In [71]:
import re
from nltk.corpus import wordnet
from random import randrange
import random
import spacy

In [72]:
bot_responses={
    'ask_username':['What should I call you?'],
    'ask_age':['How old are you?'],
    'ask_reading':['On a scale from 1 to 5, 1 being lowest and 5 highest. How much do you like reading?'],
    'starting_conv':['How was your day today?','What do you want to talk about?','Anything interesting happened with you lately?','How\'s life?','Anything you want to share with me?','Any interesting events you wanna tell me about?'],
    'ask_about':['What about {0}?','Tell me more about {0}.'],
    'normal_responses':['I understand','It\'s OK','Life is full of ups and downs'],
    'act_responses':['Maybe you should take a walk','Go to nearby park','Go to the gym']
}
intents_dict = {
    'greeting':{
        'key_words':['hello','hi','welcome'],
        'sentences':[],
        'responses':['Hi there!'],
        'patterns':[]
    },
    'fallback':{
        'key_words':[],
        'sentences':[],
        'responses':['I\'m sorry, I don\'t understand what you mean.','I dont quite understand. Could you repeat that?'],
        'patterns':[]
    },
    'gratitude':{
        'key_words':[],
        'sentences':['thank you','grateful for your help'],
        'responses':['Glad I could help','It\'s my pleasure'],
        'patterns':[]
    },
    'goodbye':{
        'key_words':['bye','quit'],
        'sentences':['bye bye','It was nice talking to you','see you'],
        'responses':['See you later.','bye bye',"I hope you enjoyed our talk! see you later."],
        'patterns':[]
    },
    'talking':{
        'key_words':[],
        'sentences':[],
        'responses':['What about {0}?','Tell me more about {0}.'],
        'patterns':[]
    }
}

default_user_profile={
    'name':'User',
    'age':'25',
    'reading':5,
    'exercise':4
}

In [73]:
#iterate on intents dictionary
for intent in intents_dict:
    word_syn=[]
    for key_word in intents_dict[intent]['key_words']:
        for syn_set in wordnet.synsets(key_word):
            for lemma in syn_set.lemmas():
                lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lemma.name())
                word_syn.append(lem_name)
    if len(word_syn) > 0:
        for synonym in set(word_syn):
            intents_dict[intent]['patterns'].append('.*\\b'+synonym+'\\b.*')
            

intents_regex_dict={}
for intent in intents_dict:
    all_aptterns = intents_dict[intent]['patterns']+intents_dict[intent]['sentences']
    if len(all_aptterns)>0:
        intents_regex_dict[intent]=re.compile('|'.join(all_aptterns))


In [74]:
def get_intent_response(intent_key,intent_dict):
    #todo: remove this hardcoded condition
    res = ''
    if intent_key != "talking":
        i = randrange(len(intent_dict[intent_key]['responses']))
        res = intent_dict[intent_key]['responses'][i]
    return res

def get_random_string(string_list):
    i = randrange(len(string_list))
    selected_str = string_list[i]
    return selected_str

def get_entities(user_input):
    ent_lst=[]
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(user_input)
    print('ner debug:'+str(doc.ents))
    for ent in doc.ents:
#         print(ent.text, ent.start_char, ent.end_char, ent.label_)
        ent_lst.append(ent.text)
    return ent_lst

def print_bot_response(response):
    if len(response):
        print('Emo-Bot: '+str(response))

user_profile=default_user_profile


def set_curr_intent(new_intent):
    print(f'setting intetn from {matched_intent} to {new_intent}')
    matched_intent = new_intent

def activate_reactive_bot_mode(intent_key,user_input):
    new_intent=intent_key
    
    if intent_key=='greeting':
        #get basic user info
        if 'name' not in user_profile.keys():
            print_bot_response(get_random_string(bot_responses['ask_username']))
            user_profile['name'] = input().lower()
        if 'age' not in user_profile.keys():
            print_bot_response(get_random_string(bot_responses['ask_age']))
            user_profile['age'] = input().lower()
        if 'reading' not in user_profile.keys():
            print_bot_response(get_random_string(bot_responses['ask_reading']))
            user_profile['reading'] = int(input().lower())
        new_intent = 'talking'
        #start conversing with user
        print_bot_response(get_random_string(bot_responses['starting_conv']))
    
    if intent_key=='talking':
        user_stroy = []
        user_stroy.append(user_input)
        #start a mini conversation to collect enough info for emotion classification
        entities_lst=get_entities(user_input)
        #todo: keep on adding intents as the user is speaking
        for ent in entities_lst:
            print('debug ner:'+ent)
            print_bot_response(get_random_string(bot_responses['ask_about']).format(ent))
            user_input = input().lower()
            user_stroy.append(user_input)
        #todo: remove useless condition
        entities_lst=[]
        if len(entities_lst)==0:
            #decide on whether the user will get a quote or emotion response
            weight_dict = {"Response": 50,"Quote": 50*user_profile['reading'],"Activity": 50}
            # random key
            key = random.choice(list(weight_dict))
            if key == "Response":
                print_bot_response(get_random_string(bot_responses['normal_responses']))
            elif key == "Activity":
                print_bot_response(get_random_string(bot_responses['act_responses']))
            else:
                #print(get_random_string(bot_responses['act_responses']))
                print_bot_response('Quote')
          
    return new_intent
        

In [75]:
get_entities('NLP subject is fun. I am studying it as a part of my professional masters in university of Ottawa')

ner debug:(NLP, Ottawa)


['NLP', 'Ottawa']

In [76]:
print ("Welcome. I'm Emo chatbot.")
# While loop to run the chatbot indefinetely
while (True):  
    # Takes the user input and converts all characters to lowercase
    user_input = input().lower()
    
    # The fallback intent is selected by default
#     matched_intent = 'fallback'
    
    for intent,pattern in intents_regex_dict.items():
        # Using the regular expression search function to look for keywords in user input
        if re.search(pattern, user_input): 
            # if a keyword matches, select the corresponding intent from the keywords_dict dictionary
            matched_intent=intent  
    
    # The chatbot prints the response that matches the selected intent
    print_bot_response (get_intent_response(matched_intent,intents_dict))
    matched_intent = activate_reactive_bot_mode(matched_intent,user_input)
    print('Intent is:'+matched_intent)
    
    if matched_intent == 'goodbye':
        break

Welcome. I'm Emo chatbot.
hi
Emo-Bot: Hi there!
Emo-Bot: What do you want to talk about?
Intent is:talking
NLP subject is fun. I am studying it as a part of my professional masters in university of Ottawa
ner debug:(nlp,)
debug ner:nlp
Emo-Bot: Tell me more about nlp.
It is about how computers can understand the human language. I like studying NLP very much.
Emo-Bot: It's OK
Intent is:talking
thank you
Emo-Bot: It's my pleasure
Intent is:gratitude
bye
Emo-Bot: bye bye
Intent is:goodbye
